# Введение в PyTorch Lightning

PyTorch Lightning — это лёгкая надстройка над **PyTorch**, которая упрощает
рутинный код обучения нейронных сетей и делает проекты чище и повторно‑используемыми.

> **Кому адресовано:** этот ноутбук рассчитан на первокурсников, которые уже знакомы
с базовым синтаксисом Python и минимально — с PyTorch, но ещё не уверенно
ориентируются в деталях обучения моделей.


## Зачем нужен PyTorch Lightning?

*  **Отделение модели от тренировки.** Вся логика обучения (оптимизация,
  логирование, сохранение checkpoints) выносится из реализации модели.
*  **Меньше «боулерплейта».** Не нужно вручную писать циклы по эпохам,
  вычисление метрик и т.д.
*  **Воспроизводимость.** Параметры тренировки и структура проекта
  стандартизированы.
*  **Инфраструктура “из коробки”.** Лёгкий переход на multi‑GPU, TPU, AMP,
  16‑bit/8‑bit precision — без изменения кода модели.


## Ключевые понятия

| Термин | Роль |
|--------|------|
| **`LightningModule`** | Класс, куда вы помещаете слои модели и определяете `forward`, `training_step`, `validation_step`, `configure_optimizers` |
| **`Trainer`** | Объект, который берёт `LightningModule` и проводит обучение / валидацию / тест |
| **`LightningDataModule`** | (Необязательно) инкапсулирует загрузку и препроцессинг данных |


## Установка

```bash
pip install pytorch-lightning torch torchvision
```

> ⚠️ Установка внутри ноутбука может занять пару минут.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torchmetrics import Accuracy
from tqdm import tqdm
import pytorch_lightning as pl

## Полное сравнение: «чистый» PyTorch vs PyTorch Lightning

Ниже два сниппета выполняют **одну и ту же задачу** — обучают простую
CNN на MNIST.

*В варианте PyTorch* мы явно описываем модель и вручную реализуем цикл по
батчам: обнуляем градиенты, считаем лосс, делаем шаг оптимизатора, выводим
метрики.

*В варианте Lightning* вся та же логика умещается в методы `LightningModule`,
а цикл обучения берёт на себя `Trainer`.

*Для наглядности рекомендуется скрыть класс моделей в обеих ячейках*

In [ ]:
# === «Чистый» PyTorch ===
class PTModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Слои сверток (Экстрактор фичей)
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.1)
        )

        # Полносвязная "голова" (Классификатор)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 10),
        )
    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Гиперпараметры и подготовка данных
lr              = 1e-4
weight_decay    = 1e-2
batch_size      = 32
num_epochs      = 3
device          = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

train_ds = MNIST(root='data', train=True, download=True, transform=transform)
val_ds   = MNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=256, shuffle=False, num_workers=4)

# Инициализация
model_pt = PTModel().to(device)
optimizer = torch.optim.AdamW(model_pt.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

best_acc = 0.0

for epoch in range(1, num_epochs + 1):
    # Обучение
    model_pt.train()
    train_loss = 0.0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} – train", leave=False):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model_pt(x)
        loss   = loss_fn(logits, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Валидация
    model_pt.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="valid", leave=False):
            x, y = x.to(device), y.to(device)

            logits = model_pt(x)
            loss   = loss_fn(logits, y)
            val_loss += loss.item()

            preds   = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total   += y.size(0)

    val_loss /= len(val_loader)
    val_acc   = correct / total

    # Лог
    print(f"[{epoch}/{num_epochs}] "
          f"train_loss={train_loss:.4f} | "
          f"val_loss={val_loss:.4f} | "
          f"val_acc={val_acc:.2%}")

    # Сохранение лучшей модели
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model_pt.state_dict(), "best_model.pt")
        print(f"  → New best model saved (acc={best_acc:.2%})")

print("Обучение завершено.")

[1/3] train_loss=0.2322 | val_loss=0.0674 | val_acc=97.88%
  → New best model saved (acc=97.88%)


[2/3] train_loss=0.0682 | val_loss=0.0456 | val_acc=98.39%
  → New best model saved (acc=98.39%)


[3/3] train_loss=0.0467 | val_loss=0.0434 | val_acc=98.67%
  → New best model saved (acc=98.67%)
Обучение завершено.


## Та же самая логика c PyTorch Lightning

Обратите внимание, что мы **не пишем** цикл по батчам/эпохам — за нас это
делает `Trainer`.


In [2]:
# === PyTorch Lightning ===
class LitMNIST(pl.LightningModule):
    def __init__(self, lr: float = 1e-3, weight_decay: float = 1e-2):
        super().__init__()
        self.save_hyperparameters()  # сохраняем гиперпараметры
        
        # Слои сверток (Экстрактор фичей)
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.1),
        )

        # Полносвязная "голова" (Классификатор)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10),
        )

        # Функция потерь
        self.loss_fn = nn.CrossEntropyLoss()

        # Метрика
        self.val_acc = Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        preds = torch.argmax(logits, dim=1)     # Получаем предсказанные классы
        acc = self.val_acc(preds, y)

        # prog_bar=True выводит метрики в прогресс-баре Trainer-а. Note: недоступно в ipynb
        self.log("val_loss", loss, prog_bar=False, on_epoch=True, on_step=False)
        self.log("val_acc", acc,  prog_bar=False, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay,
        )
        return optimizer
    
# Датасет и DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

mnist_train = MNIST(root="data", train=True, download=True, transform=transform)
mnist_val   = MNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(mnist_train, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(mnist_val,   batch_size=256, shuffle=False, num_workers=4)

# Инициализация модели и тренера
model = LitMNIST(lr=1e-4, weight_decay=1e-2)
trainer = pl.Trainer(
    max_epochs=3, 
    accelerator="gpu",  
    devices=1,                  # Выбираем кол-во видеокарт
    # strategy="ddp",           # Если реально несколько GPU и доступна DDP. Note: DDP недоступна в ipynb
    log_every_n_steps=50,
)

# Обучение
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | conv    | Sequential         | 18.8 K | train
1 | fc      | Sequential         | 1.2 M  | train
2 | loss_fn | CrossEntropyLoss   | 0      | train
3 | val_acc | MulticlassAccuracy | 0      | train
-------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.800     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Логирование и мониторинг: TensorBoard и Weights & Biases

PyTorch Lightning поддерживает популярные системы отслеживания экспериментов
«из коробки». Достаточно передать объект‑логгер в `Trainer`. Метрики,
графики лосса и другие параметры будут автоматически
отправляться в выбранный backend.

* **TensorBoard** — локальный веб‑интерфейс, входящий в состав PyTorch.
* **Weights & Biases (wandb)** — облачный сервис, удобный для коллаборации.

```python
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger

# TensorBoard (метрики сохранятся в ./tb_logs/)
tb_logger = TensorBoardLogger("tb_logs", name="mnist")

# wandb (сначала `pip install wandb`)
# wb_logger = WandbLogger(project="mnist-demo")
```

> 💡 Если wandb не установлен, Python пропустит создание `WandbLogger`.


### Запуск TensorBoard после начала обучения

> **Важно:** по умолчанию `pytorch_lightning` сохраняет логи в папку `lightning_logs/`.

1. **Внутри ноутбука**  
   ```python
   %load_ext tensorboard
   %tensorboard --logdir lightning_logs/ --port 6006
   ```
   После выполнения под ячейкой появится интерфейс TensorBoard и графики будут обновляться в режиме реального времени.

2. **Через терминал**  
   Откройте терминал в папке с ноутбуком и выполните:  
   ```bash
   tensorboard --logdir lightning_logs/ --port 6006
   ```
   Затем перейдите в браузере по адресу <http://localhost:6006>.

3. **Если обучение идёт на удалённом сервере**  
   Пробросьте порт:
   ```bash
   ssh -L 6006:localhost:6006 user@remote-server
   ```
   После этого откройте `http://localhost:6006` в своём браузере.

> **Советы**  
> * Меняйте номер порта (`--port 6007`), если 6006 уже занят.  
> * Для сравнения экспериментов сохраняйте логи в разные подпапки, например `run1/`, `run2/`.

In [3]:
from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.loggers import WandbLogger

tb_logger = TensorBoardLogger("tb_logs", name="mnist")

trainer = pl.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    # strategy="ddp",
    logger=tb_logger,   # Передаем объект логгера: 'logger=tb_logger'
    log_every_n_steps=50
)

# Обучение
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | conv    | Sequential         | 18.8 K | train
1 | fc      | Sequential         | 1.2 M  | train
2 | loss_fn | CrossEntropyLoss   | 0      | train
3 | val_acc | MulticlassAccuracy | 0      | train
-------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.800     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ --port 6006
# Далее открываем localhost:6006 в своем браузере. Note: VPN может мешать

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 20004), started 0:03:39 ago. (Use '!kill 20004' to kill it.)

## Когда применять Lightning?

* **Учебные и исследовательские проекты.** Сфокусируйтесь на идеях, а не
  на вспомогательном коде.
* **Командная разработка.** Стандартизированная структура облегчает ревью и
  переиспользование.
* **Прототипы с возможностью масштабирования.** Когда понадобится
  распределённое обучение — код уже готов.


## Полезные ссылки

* **Обязательно к изучению:** <https://lightning.ai/docs/pytorch/stable/starter/introduction.html>
* Документация: <https://lightning.ai/docs/pytorch/latest/>
* Шорт‑гайды: <https://lightning.ai/docs/pytorch/stable/tutorials.html>
* GitHub: <https://github.com/Lightning-AI/pytorch-lightning>


## P.S.
* Made by REU DS Club
* По всем вопросам в tg: @yoursAnthony
## Links (REU DS Club):
* vk: https://vk.com/reu_ds_club
* tg: https://t.me/+pkbsRGH1Bt030DNi